In [67]:
import os 
import numpy as np
import pandas  as pd

In [68]:
current_path =  os.getcwd()
prediction_file_path =  'prediction_lgbm.csv'
prediction_path = os.path.join(current_path, prediction_file_path)
print(f'The path file for the train dataset is: {prediction_path}')

The path file for the train dataset is: C:\Users\Admin\Desktop\finally-datascience\prediction_lgbm.csv


In [69]:
prediction_df = pd.read_csv(prediction_path).reset_index(drop=True)

In [70]:
prediction_df.head(5)

,fecha,total_calls
0,2018-10-01,25946.483415
1,2018-10-02,29357.264266
2,2018-10-03,29267.850762
3,2018-10-04,27939.861661
4,2018-10-05,26683.139977


In [71]:
current_path =  os.getcwd()
conf_day_hors_df_file_path =  'configuration_days_hours.csv'
conf_day_hors_df_path = os.path.join(current_path, conf_day_hors_df_file_path)
print(f'The path file for the train dataset is: {conf_day_hors_df_path}')

The path file for the train dataset is: C:\Users\Admin\Desktop\finally-datascience\configuration_days_hours.csv


In [72]:
conf_day_hors_df = pd.read_csv(conf_day_hors_df_path).reset_index(drop=True)

In [73]:
conf_day_hors_df.head(7)

,0,1,2,3,4,5,6,7,8,9,10
0,6.4,10.5,11.1,11.0,10.1,9.3,8.9,9.0,9.2,8.4,6.1
1,6.9,10.1,10.0,10.6,10.0,9.0,8.6,8.7,9.6,9.3,6.3
2,6.4,9.8,10.8,10.8,10.2,9.2,8.9,9.1,9.3,8.9,6.4
3,6.4,9.7,10.9,10.7,10.0,9.2,8.6,9.2,9.7,9.5,6.1
4,6.6,10.0,10.9,10.8,10.0,9.2,8.9,9.1,9.5,9.0,6.0
5,8.1,12.3,14.4,14.6,12.7,9.1,8.1,7.7,6.9,6.0,0.1


In [74]:
conf_day_hors_df.iloc[0]

0      6.4
1     10.5
2     11.1
3     11.0
4     10.1
5      9.3
6      8.9
7      9.0
8      9.2
9      8.4
10     6.1
Name: 0, dtype: float64

In [75]:
"""
return predictions_cube[date][dayOfWeek][hour]
"""
def prediction_table(predictions: pd.DataFrame, distribution: pd.DataFrame, ignoreDays={6,}):
    predictions['fecha'] = pd.to_datetime(predictions['fecha'])
    predictions['day_of_week'] = predictions['fecha'].dt.dayofweek
    numb_days : int = predictions.shape[0]
    numb_dayOfWeek : int = distribution.shape[0]
    numb_hours : int = distribution.shape[1]
    
    predictions_cube = np.zeros(shape=(numb_days,numb_hours))
    predictions_arr = predictions.values
    distribution_arr = distribution.values
    
    for pred in range(0, numb_days):
        dayOfWeek = predictions_arr[pred][2]
        for hour in range(0, numb_hours):
            if dayOfWeek in ignoreDays:
                predictions_cube[pred][hour] = 0
            else: 
                predictions_cube[pred][hour] = predictions_arr[pred][1] * distribution_arr[dayOfWeek][hour] / 100
            
    return predictions_cube


In [76]:
predictions_cube = prediction_table(predictions= prediction_df, distribution= conf_day_hors_df )

In [77]:
fecha_inicio = pd.to_datetime('2018-10-01')
rango_fechas = pd.date_range(start=fecha_inicio, periods=30, freq='D') # 30 dias
predicciones_distribucion = pd.DataFrame(predictions_cube, index=rango_fechas)
predicciones_distribucion.columns = [f'Hora_{i}' for i in range(11)] # 11 horas


In [78]:
predicciones_distribucion

,Hora_0,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,Hora_7,Hora_8,Hora_9,Hora_10
2018-10-01,1660.574939,2724.380759,2880.059659,2854.113176,2620.594825,2413.022958,2309.237024,2335.183507,2387.076474,2179.504607,1582.735488
2018-10-02,2025.651234,2965.083691,2935.726427,3111.870012,2935.726427,2642.153784,2524.724727,2554.081991,2818.297370,2730.225577,1849.507649
2018-10-03,1873.142449,2868.249375,3160.927882,3160.927882,2985.320778,2692.642270,2604.838718,2663.374419,2721.910121,2604.838718,1873.142449
2018-10-04,1788.151146,2710.166581,3045.444921,2989.565198,2793.986166,2570.467273,2402.828103,2570.467273,2710.166581,2654.286858,1704.331561
2018-10-05,1761.087239,2668.313998,2908.462258,2881.779118,2668.313998,2454.848878,2374.799458,2428.165738,2534.898298,2401.482598,1600.988399
2018-10-06,2036.690764,3092.752641,3620.783580,3671.072241,3193.329963,2288.134068,2036.690764,1936.113442,1734.958799,1508.659825,25.144330
2018-10-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-10-08,1685.850371,2765.848265,2923.896738,2897.555326,2660.482617,2449.751321,2344.385672,2370.727085,2423.409909,2212.678612,1606.826135
2018-10-09,1966.486354,2878.480025,2849.980223,3020.979036,2849.980223,2564.982201,2450.982992,2479.482794,2735.981014,2650.481607,1795.487541
2018-10-10,1839.785645,2817.171769,3104.638276,3104.638276,2932.158372,2644.691865,2558.451913,2615.945214,2673.438516,2558.451913,1839.785645


In [81]:
predicciones_distribucion.to_excel("./distribucion_prediccion_hora.xlsx", index=False)